In [ ]:
'''
Project Overview[Payment Fraud Risk Detection in Online Transactions]:

Goals:To investigate transaction patterns to identify potential fraud.
To create an initial risk assessment methodology using transaction characteristics...

'''

In [4]:
# Importing necessary libraries for analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Getting the Dataset
fct_transactions= pd.read_csv(r'D:\Data Journey\Python-Summer-Party\DataSets\fct_transactions1.csv')

# Display the first few rows to understand the data
print(fct_transactions.head(10))  # Shows the first 5 rows with columns
print(list(fct_transactions.columns))
print('Number of rows and columns is:', fct_transactions.shape)

         customer_email  transaction_id transaction_date  transaction_amount  \
0       alice@gmail.com               1        10/5/2024              120.00   
1  bob@customdomain.com               2       10/15/2024              250.50   
2     charlie@yahoo.com               3       10/20/2024               75.25   
3      dana@hotmail.com               4       10/25/2024              100.00   
4           eve@biz.org               5       10/30/2024              300.00   
5       frank@gmail.com               6        11/3/2024              150.75   
6     grace@outlook.com               7       11/10/2024                 NaN   
7        ivan@yahoo.com               8       11/15/2024              200.00   
8      judy@hotmail.com               9       11/21/2024              250.00   
9        ken@domain.net              10       11/29/2024              300.00   

   fraud_detection_score  
0                     10  
1                     20  
2                     15  
3          

In [22]:
# Question One
# Transactions in October 2024 have a customer email ending with a domain other than 'gmail.com', 'yahoo.com', or 'hotmail.com'
# 0. Changing the transaction date to appropriate date column
fct_transactions['transaction_date']= pd.to_datetime(fct_transactions['transaction_date'])
# Subsetting for July 2024 only
trxn_Oct_2024 = fct_transactions[(fct_transactions['transaction_date'].dt.year == 2024) & (fct_transactions['transaction_date'].dt.month == 10)]
# Make sure you're working on a copy, not a slice
trxn_Oct_2024 = trxn_Oct_2024.copy()
accepted_domains = ['gmail.com', 'yahoo.com', 'hotmail.com']
# Extract domain from email
trxn_Oct_2024['email_domain'] = trxn_Oct_2024['customer_email'].str.split('@').str[-1]

# Check if domain is in the accepted list
trxn_Oct_2024['is_accepted'] = trxn_Oct_2024['email_domain'].isin(accepted_domains)

# Count transactions with unaccepted domains
unaccepted_count = (~trxn_Oct_2024['is_accepted']).sum()

print(f"Number of transactions in October 2024 with unaccepted domains: {unaccepted_count}")


Number of transactions in October 2024 with unaccepted domains: 2
